In [1]:
!pip install ktrain

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from ktrain.text.zsl import ZeroShotClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Fing the first category from the prpbability

In [4]:
def find_max_categories_for_lists(lists_of_category_scores):
    max_categories = []
    for category_scores in lists_of_category_scores:
        max_category = max(category_scores, key=lambda x: x[1])[0]
        max_categories.append(max_category)
    return max_categories

# Find the maximum first 2 categories fro mthe probability

In [5]:
def find_max_2_categories_for_lists(lists_of_category_scores):
    max_categories = []
    for category_scores in lists_of_category_scores:
        # Sort the categories based on their values in descending order
        sorted_categories = sorted(category_scores, key=lambda x: x[1], reverse=True)
        # Select the top two categories
        top_categories = [category[0] for category in sorted_categories[:2]]
        max_categories.append(top_categories)
    return max_categories

## check for the best category from the first 2 categories depending on the label of this data
if any of the 2 values are equal to the true label then return it


if none of them are equal to the true label then return the first value with the highest probability

In [6]:
def return_best_category_from_2_categories(list1, list2):
  result = []
  for item1, item2 in zip(list1, list2):
      found = False
      for value in item2:
          if value in item1:
              result.append(value)
              found = True
              break
      if not found:
          result.append(item2[0])
  return result


In [7]:
def compute_classification_report(y_true, y_pred):

    report = classification_report(y_true, y_pred)
    print(report)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [8]:
def find_max_value(category_scores):
    max_value = max(category_scores, key=lambda x: x[1])[1]
    return max_value

In [27]:
df = pd.read_csv("https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/TOPICS/DATA/MN-DS-news-classification.csv")
df = df.rename(columns={'category_level_1': 'category_1', 'category_level_2': 'category_2'})
df

,data_id,id,date,source,title,content,author,url,published,published_utc,collection_utc,category_1,category_2
0,1809,abcnews--2019-10-31--Virginia mom charged with...,2019-10-31,abcnews,Virginia mom charged with murder in 2-year-old...,The Virginia woman whose 2-year-old son was fo...,NaN,https://abcnews.go.com/US/wireStory/virginia-m...,"Thu, 31 Oct 2019 16:49:56 -0400",1572554996,1572559512,"crime, law and justice",crime
1,1980,abcnews--2019-11-07--2 escaped murder suspects...,2019-11-07,abcnews,2 escaped murder suspects arrested at US-Mexic...,Authorities are trying to determine if anyone ...,NaN,https://abcnews.go.com/US/wireStory/escaped-mu...,"Thu, 07 Nov 2019 00:13:12 -0500",1573103592,1573131986,"crime, law and justice",crime
2,1995,abcnews--2019-11-07--Family turns in escaped b...,2019-11-07,abcnews,"Family turns in escaped boy, 13, suspected in ...",A 13-year-old suspect in a double homicide who...,NaN,https://abcnews.go.com/US/wireStory/family-tur...,"Thu, 07 Nov 2019 07:39:54 -0500",1573130394,1573131982,"crime, law and justice",crime
3,2740,abcnews--2019-12-02--Mother charged with murde...,2019-12-02,abcnews,Mother charged with murder in deaths of 2 youn...,The mother of two young children found hanging...,NaN,https://abcnews.go.com/US/wireStory/mother-cha...,"Mon, 02 Dec 2019 11:30:59 -0500",1575304259,1575308811,"crime, law and justice",crime
4,7038,ageofautism--2019-04-12--Physician Father and ...,2019-04-12,ageofautism,"Physician, Father and Caretaker of 29 Year Old...","""One family member said Derek “can be violent ...",Age of Autism,http://feedproxy.google.com/~r/ageofautism/~3/...,2019-04-12 09:00:00+00:00,1555074000,1567543083,"crime, law and justice",crime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10912,907640,therussophileorg--2019-12-15--Iran to build ov...,2019-12-15,therussophileorg,"Iran to build over 30,000 housing units in Syr...",This post was originally published on this sit...,News Desk,https://www.therussophile.org/iran-to-build-ov...,"Sun, 15 Dec 2019 10:42:50 +0000",1576424570,1576413702,"conflict, war and peace",post-war reconstruction
10913,892720,therussophileorg--2019-10-12--Ukraine has no m...,2019-10-12,therussophileorg,Ukraine has no money for reconstruction of Don...,This post was originally published on this sit...,Michael Sullivan,https://www.therussophile.org/ukraine-has-no-m...,"Sat, 12 Oct 2019 20:56:01 +0000",1570928161,1570922006,"conflict, war and peace",post-war reconstruction
10914,870499,therussophileorg--2019-07-06--Irans contributi...,2019-07-06,therussophileorg,Iran’s contribution is needed to rebuild Iraq ...,This\n\n[post](http://www.presstv.ir/Detail/20...,Michael Sullivan,https://www.therussophile.org/irans-contributi...,2019-07-06 17:51:38+00:00,1562449898,1567536625,"conflict, war and peace",post-war reconstruction
10915,887334,therussophileorg--2019-09-25--Iraqi president ...,2019-09-25,therussophileorg,Iraqi president demands international help for...,This\n\n[post](http://www.presstv.ir/Detail/20...,Michael Sullivan,https://www.therussophile.org/iraqi-president-...,2019-09-25 16:58:01+00:00,1569445081,1570222223,"conflict, war and peace",post-war reconstruction


In [60]:
df.content[0]

"The Virginia woman whose 2-year-old son was found in a trash incinerator has been charged with murder in his death.\n\nHampton Commonwealth's Attorney Anton Bell told a news conference on Thursday that a warrant had been issued for 34-year-old Julia Leanna Tomlin, who also will be charged with unlawfully disposing of Noah Tomlin's body.\n\nBell said skull fractures found on the toddler's body indicates a level of force so severe it was as if the child had fallen several stories from a building.\n\nJulia Tomlin reported her son missing in June, and searchers sifted through a landfill and steam plant over 10 days before the body was found on July 3. Tomlin is already jailed and charged with three counts of felony child neglect prior to reporting Noah missing."

In [50]:
X = df.content
Y = df.category_1

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X,Y , shuffle=True, test_size = 100, random_state = 42 , stratify=Y)

In [65]:
X_test.shape

(100,)

In [52]:
zsl = ZeroShotClassifier()
labels=['arts, culture, entertainment and media',
       'conflict, war and peace', 'crime, law and justice',
       'disaster, accident and emergency incident',
       'economy, business and finance', 'education', 'environment',
       'health', 'human interest', 'labour', 'lifestyle and leisure',
       'politics', 'religion and belief', 'science and technology',
       'society', 'sport', 'weather']

In [61]:
doc = """The Virginia woman whose 2-year-old son was found in a trash incinerator has been charged with murder in his death.

Hampton Commonwealth's Attorney Anton Bell told a news conference on Thursday that a warrant had been issued for 34-year-old Julia Leanna Tomlin, who also will be charged with unlawfully disposing of Noah Tomlin's body.

Bell said skull fractures found on the toddler's body indicates a level of force so severe it was as if the child had fallen several stories from a building.

Julia Tomlin reported her son missing in June, and searchers sifted through a landfill and steam plant over 10 days before the body was found on July 3. Tomlin is already jailed and charged with three counts of felony child neglect prior to reporting Noah missing."""
# trn = train["Excerpt"].iloc[:100].values
# y_trn = train["Category"].iloc[:100].values
# tst = test["Excerpt"].values
# y_tst = test["Category"].values

In [ ]:
%%time
doc_prb = zsl.predict(X_test.values, labels=labels, include_labels=True , batch_size = 16)

In [63]:
doc_prb

[('arts, culture, entertainment and media', 0.15724362432956696),
 ('conflict, war and peace', 0.03940753638744354),
 ('crime, law and justice', 0.8896970748901367),
 ('disaster, accident and emergency incident', 0.7422193884849548),
 ('economy, business and finance', 0.090092733502388),
 ('education', 0.00984184443950653),
 ('environment', 0.1493915319442749),
 ('health', 0.14030632376670837),
 ('human interest', 0.4270774722099304),
 ('labour', 0.0837382823228836),
 ('lifestyle and leisure', 0.019998442381620407),
 ('politics', 0.05654836818575859),
 ('religion and belief', 0.03391776233911514),
 ('science and technology', 0.1140076220035553),
 ('society', 0.4139717221260071),
 ('sport', 0.05076543986797333),
 ('weather', 0.07070475071668625)]

In [42]:
df2_pred = find_max_categories_for_lists(doc_prb)
# y_trn_pred = find_max_categories_for_lists(y_trn_prb)
# y_tst_pred = find_max_categories_for_lists(y_tst_prb)

In [43]:
compute_classification_report(y_test , df2_pred)

                                           precision    recall  f1-score   support

   arts, culture, entertainment and media       1.00      0.38      0.55        21
                  conflict, war and peace       0.00      0.00      0.00         0
                   crime, law and justice       0.94      0.78      0.85        40
disaster, accident and emergency incident       0.60      0.60      0.60        10
            economy, business and finance       1.00      0.38      0.55        29
                              environment       0.00      0.00      0.00         0
                                   health       0.00      0.00      0.00         0
                           human interest       0.00      0.00      0.00         0
                                   labour       0.00      0.00      0.00         0
                                 politics       0.00      0.00      0.00         0
                                  society       0.00      0.00      0.00         0
   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.56

In [20]:
compute_classification_report(y_trn , y_trn_pred)

               precision    recall  f1-score   support

     business       0.76      0.87      0.81        30
entertainment       1.00      0.67      0.80         9
       health       0.62      0.71      0.67         7
     politics       0.91      0.81      0.86        26
       sports       1.00      0.86      0.93        22
         tech       0.50      0.83      0.62         6

     accuracy                           0.82       100
    macro avg       0.80      0.79      0.78       100
 weighted avg       0.85      0.82      0.83       100



0.82

In [21]:
compute_classification_report(y_tst , y_tst_pred)

               precision    recall  f1-score   support

     business       0.77      0.83      0.80       224
entertainment       0.95      0.81      0.87        67
       health       0.86      0.82      0.84        97
     politics       0.89      0.89      0.89       190
       sports       0.98      0.84      0.91       199
         tech       0.46      0.67      0.54        51

     accuracy                           0.84       828
    macro avg       0.82      0.81      0.81       828
 weighted avg       0.85      0.84      0.84       828



0.8357487922705314

# After getting the 2 maximum probability categories i check if **the first or the second** category is the same as true label and get the classification report

In [22]:
df2_pred = find_max_2_categories_for_lists(df2_prb)
y_trn_pred = find_max_2_categories_for_lists(y_trn_prb)
y_tst_pred = find_max_2_categories_for_lists(y_tst_prb)

In [23]:
df2_pred = return_best_category_from_2_categories(val , df2_pred)
y_trn_pred = return_best_category_from_2_categories(y_trn , y_trn_pred)
y_tst_pred = return_best_category_from_2_categories(y_tst , y_tst_pred)

In [24]:
compute_classification_report(val , df2_pred)

              precision    recall  f1-score   support

    Business       0.97      0.87      0.92        77
    Sci/Tech       0.86      0.78      0.82        23
      Sports       0.00      0.00      0.00         0
       World       0.00      0.00      0.00         0

    accuracy                           0.85       100
   macro avg       0.46      0.41      0.43       100
weighted avg       0.94      0.85      0.89       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.85

In [25]:
compute_classification_report(y_trn , y_trn_pred)

               precision    recall  f1-score   support

     business       0.88      0.97      0.92        30
entertainment       1.00      0.78      0.88         9
       health       0.83      0.71      0.77         7
     politics       0.93      0.96      0.94        26
       sports       1.00      0.95      0.98        22
         tech       1.00      1.00      1.00         6

     accuracy                           0.93       100
    macro avg       0.94      0.90      0.91       100
 weighted avg       0.93      0.93      0.93       100



0.93

In [26]:
compute_classification_report(y_tst , y_tst_pred)

               precision    recall  f1-score   support

     business       0.94      0.99      0.97       224
entertainment       0.98      0.93      0.95        67
       health       0.97      0.89      0.92        97
     politics       0.98      0.97      0.98       190
       sports       0.99      0.98      0.99       199
         tech       0.89      0.96      0.92        51

     accuracy                           0.96       828
    macro avg       0.96      0.95      0.96       828
 weighted avg       0.97      0.96      0.96       828



0.964975845410628